In [3]:
%cd /kaggle/working/

/kaggle/working


In [4]:
!git clone https://github.com/VarunGumma/IndicTransTokenizer

Cloning into 'IndicTransTokenizer'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 123 (delta 52), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (123/123), 3.85 MiB | 13.56 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [5]:
%cd IndicTransTokenizer

/kaggle/working/IndicTransTokenizer


In [6]:
pip install --editable ./

Obtaining file:///kaggle/working/IndicTransTokenizer
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-212qjmqf/indic-nlp-library-it2_41d523a7a8594908bbf64d317262efb1
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-212qjmqf/indic-nlp-library-it2_41d523a7a8594908bbf64d317262efb1
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 09d30a15286cc252a12682e5450c807379717eaf
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 894.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 1.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.1 MB/s eta 0:0

In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

tokenizer = IndicTransTokenizer(direction="en-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-en-indic-dist-200M", trust_remote_code=True, low_cpu_mem_usage=True)
model=model.to("cuda")
# def translate_from_file(input_file, hindi_to_english):

#   with open(input_file, 'r') as f:
#     sentences = f.readlines()

#   batch = ip.preprocess_batch(sentences, src_lang="eng_Latn", tgt_lang="hin_Deva")
#   batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

#   with torch.inference_mode():
#     outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=256)

#   outputs = tokenizer.batch_decode(outputs, src=False)
#   outputs = ip.postprocess_batch(outputs, lang="hin_Deva")

#   hindi_to_english.extend(outputs)  # Append translated sentences to the output list

# # Example usage:
# input_file = "/kaggle/input/extracted-22/extracted_eng.txt"  # Replace with your actual file path
# hindi_to_english = []

# translate_from_file(input_file, hindi_to_english)

# print(output_list)  # List containing the translated sentences from the file


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [9]:
translation = []
file_path = "/kaggle/input/extracted-22/extracted_eng.txt"  
with open(file_path, "r", encoding="utf-8") as file:
    input_sentences = [line.strip() for line in file]
BATCH_SIZE=4
for i in range(0,len(input_sentences),BATCH_SIZE):
    batch = input_sentences[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="eng_Latn", tgt_lang="hin_Deva")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)
    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
    translation.append(outputs)
    del batch
    torch.cuda.empty_cache()

In [10]:
import torch
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

tokenizer = IndicTransTokenizer(direction="indic-en")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-en-dist-200M", trust_remote_code=True, low_cpu_mem_usage=True)
model=model.to("cuda")
# def translate_from_file(input_file, hindi_to_english):

#   with open(input_file, 'r') as f:
#     sentences = f.readlines()

#   batch = ip.preprocess_batch(sentences, src_lang="eng_Latn", tgt_lang="hin_Deva")
#   batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

#   with torch.inference_mode():
#     outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=256)

#   outputs = tokenizer.batch_decode(outputs, src=False)
#   outputs = ip.postprocess_batch(outputs, lang="hin_Deva")

#   hindi_to_english.extend(outputs)  # Append translated sentences to the output list

# # Example usage:
# input_file = "/kaggle/input/extracted-22/extracted_eng.txt"  # Replace with your actual file path
# hindi_to_english = []

# translate_from_file(input_file, hindi_to_english)

# print(output_list)  # List containing the translated sentences from the file


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/914M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [11]:
hindi_to_english = []
file_path = "/kaggle/input/extracted-22/extracted_hi.txt"  
with open(file_path, "r", encoding="utf-8") as file:
    input_sentences = [line.strip() for line in file]
BATCH_SIZE=4
for i in range(0,len(input_sentences),BATCH_SIZE):
    batch = input_sentences[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="hin_Deva", tgt_lang="eng_Latn")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)
    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="eng_Latn")
    hindi_to_english.append(outputs)
    del batch
    torch.cuda.empty_cache()

In [12]:
import torch
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

tokenizer = IndicTransTokenizer(direction="indic-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-indic-dist-320M", trust_remote_code=True, low_cpu_mem_usage=True)
model=model.to("cuda")
# def translate_from_file(input_file, hindi_to_english):

#   with open(input_file, 'r') as f:
#     sentences = f.readlines()

#   batch = ip.preprocess_batch(sentences, src_lang="eng_Latn", tgt_lang="hin_Deva")
#   batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

#   with torch.inference_mode():
#     outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=256)

#   outputs = tokenizer.batch_decode(outputs, src=False)
#   outputs = ip.postprocess_batch(outputs, lang="hin_Deva")

#   hindi_to_english.extend(outputs)  # Append translated sentences to the output list

# # Example usage:
# input_file = "/kaggle/input/extracted-22/extracted_eng.txt"  # Replace with your actual file path
# hindi_to_english = []

# translate_from_file(input_file, hindi_to_english)

# print(output_list)  # List containing the translated sentences from the file


config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [13]:
hindi_to_gujrati = []
file_path = "/kaggle/input/extracted-22/extracted_hi.txt"  
with open(file_path, "r", encoding="utf-8") as file:
    input_sentences = [line.strip() for line in file]
BATCH_SIZE=4
for i in range(0,len(input_sentences),BATCH_SIZE):
    batch = input_sentences[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="hin_Deva", tgt_lang="guj_Gujr")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)
    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="guj_Gujr")
    hindi_to_gujrati.append(outputs)
    del batch
    torch.cuda.empty_cache()

In [14]:
import torch
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

tokenizer = IndicTransTokenizer(direction="indic-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-indic-dist-320M", trust_remote_code=True, low_cpu_mem_usage=True)
model=model.to("cuda")
# def translate_from_file(input_file, hindi_to_english):

#   with open(input_file, 'r') as f:
#     sentences = f.readlines()

#   batch = ip.preprocess_batch(sentences, src_lang="eng_Latn", tgt_lang="hin_Deva")
#   batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

#   with torch.inference_mode():
#     outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=256)

#   outputs = tokenizer.batch_decode(outputs, src=False)
#   outputs = ip.postprocess_batch(outputs, lang="hin_Deva")

#   hindi_to_english.extend(outputs)  # Append translated sentences to the output list

# # Example usage:
# input_file = "/kaggle/input/extracted-22/extracted_eng.txt"  # Replace with your actual file path
# hindi_to_english = []

# translate_from_file(input_file, hindi_to_english)

# print(output_list)  # List containing the translated sentences from the file


In [15]:
gujrati_to_hindi = []
file_path = "/kaggle/input/extracted-22/extracted_guj2.txt"  
with open(file_path, "r", encoding="utf-8") as file:
    input_sentences = [line.strip() for line in file]
BATCH_SIZE=4
for i in range(0,len(input_sentences),BATCH_SIZE):
    batch = input_sentences[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="guj_Gujr", tgt_lang="hin_Deva")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)
    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
    gujrati_to_hindi.append(outputs)
    del batch
    torch.cuda.empty_cache()

In [19]:
file_name = "engg_to_hin_1B.txt"  # Add ".txt" extension to make it a text file

with open(file_name, "w", encoding="utf-8") as file:
    for sentences in translation:
        for sentence in sentences:
            file.write(sentence + "\n")

# print("Sentences have been written to:", file_name)

# No need to explicitly save the file as it's already done when writing.


In [22]:
file_name = "guj_to_hin_1B.txt"  # Add ".txt" extension to make it a text file

with open(file_name, "w", encoding="utf-8") as file:
    for sentences in gujrati_to_hindi:
        for sentence in sentences:
            file.write(sentence + "\n")

# print("Sentences have been written to:", file_name)

# No need to explicitly save the file as it's already done when writing.
